In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import re
#re.compile('<title>(.*)</title>')
import torch
import torch.nn as nn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle

In [2]:
import pandas as pd

# Load the Excel file
file_path = 'https://docs.google.com/spreadsheets/d/12oPDLqEbwcCN1bhzfFIAkOFRU0Vqf7xY/export?format=xlsx'  # Update this path to the location of your uploaded file
sheet_name = 'First Data'

# Read the specific sheet into a DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Display the DataFrame
df.head()


,Question,Answer
0,"Ẹ káàbọ̀, sà. Kí ni mo lè ṣe fún yín?",Mo fẹ́ẹ́ ra àkàrà-òyìnbó fún ọjọ́-ìbí ni
1,Ó dáa. Onírú èròja lẹ fẹ́ẹ́ rà?,Onípele méjì ti ó ní èròjà ‘strawberry’ ni
2,Ṣé àkàrà-òyìnbó aláìlẹ́yin lẹ fẹ́?,"Bẹ́ẹ̀ni, aláìlẹ́yin ni mo fẹ́."
3,"Ó dáa. Ṣùgbọn, ẹni tó bá béèrè ní a máa ń ṣe a...","Kò burú. Màá padà wá títí aago méjì, ṣùgbọ́n è..."
4,Báwo ni kí àkàrà-òyìnbó náà tóbi tó?,Oní-kílò mẹ́ta.


In [3]:
  #pip install --ignore-installed torch --user

In [4]:
#pip install torch

In [5]:
# total amount of rows and columns
total_rows=len(df.axes[0])
print("total rows", total_rows)
total_cols=len(df.axes[1])
print("total columns", total_cols)

total rows 209
total columns 2


In [6]:
#Case conversion
def lower(text):
    #convert all the text into lower letters
    #remove all special characters: except space ''
    text =text.lower()
    #text = re.sub('[^A-Za-z]+','',text)
    return text

In [7]:
df['Question'] = df['Question'].apply(lower)
df['Answer'] = df['Answer'].astype(str).apply(lower)

In [8]:
df.head(10)

,Question,Answer
0,"ẹ káàbọ̀, sà. kí ni mo lè ṣe fún yín?",mo fẹ́ẹ́ ra àkàrà-òyìnbó fún ọjọ́-ìbí ni
1,ó dáa. onírú èròja lẹ fẹ́ẹ́ rà?,onípele méjì ti ó ní èròjà ‘strawberry’ ni
2,ṣé àkàrà-òyìnbó aláìlẹ́yin lẹ fẹ́?,"bẹ́ẹ̀ni, aláìlẹ́yin ni mo fẹ́."
3,"ó dáa. ṣùgbọn, ẹni tó bá béèrè ní a máa ń ṣe a...","kò burú. màá padà wá títí aago méjì, ṣùgbọ́n è..."
4,báwo ni kí àkàrà-òyìnbó náà tóbi tó?,oní-kílò mẹ́ta.
5,á jẹ́ ẹgbẹ̀rún méje náírà (₦7000),ẹ̀dínwó ńkọ́?
6,"ẹ má bínú, ọ̀gá, j́alẹ̀jálẹ̀ ni mo ti sọ fún yín.","á-àá, èmi tí mo ti jẹ́ oníbàárà yín tipẹ́. ó y..."
7,"ẹ má bínú, ọ̀gá, ṣùgbọ́n n kò lè fún yíin ní ẹ...",ó dáa. màá fún ẹ ní ẹgbẹ̀rún méje náírà (₦700...
8,ẹ fọkànbalẹ̀. ẹ̀yin náà sáà mọ̀ wá. adùn ti wa...,ó dáa. màá wá ní aago méjì. mo fẹ́ kí ṣọnà òdò...
9,ìyẹn kò ṣòro. ẹ máa san ìdajì owó ní àsansílẹ̀.,"á-àá, ṣebí oníbàárà tipẹ́tipẹ́ ni mí. ẹ ò lè f..."


In [9]:
#Removal of special characters
def remove_special_characters (df, column, characters):
    for char in characters:
        df[column]= df[column].str.replace(char,'',regex=False)
    return df
#special characters to remove
special_characters = ['?','#','(',')',',','₦','!','-','N','']
#Remove special characters from columns
df = remove_special_characters(df, 'Question',special_characters)
df = remove_special_characters(df, 'Answer',special_characters)


In [10]:
df.head(10)

,Question,Answer
0,ẹ káàbọ̀ sà. kí ni mo lè ṣe fún yín,mo fẹ́ẹ́ ra àkàràòyìnbó fún ọjọ́ìbí ni
1,ó dáa. onírú èròja lẹ fẹ́ẹ́ rà,onípele méjì ti ó ní èròjà ‘strawberry’ ni
2,ṣé àkàràòyìnbó aláìlẹ́yin lẹ fẹ́,bẹ́ẹ̀ni aláìlẹ́yin ni mo fẹ́.
3,ó dáa. ṣùgbọn ẹni tó bá béèrè ní a máa ń ṣe al...,kò burú. màá padà wá títí aago méjì ṣùgbọ́n èl...
4,báwo ni kí àkàràòyìnbó náà tóbi tó,oníkílò mẹ́ta.
5,á jẹ́ ẹgbẹ̀rún méje náírà 7000,ẹ̀dínwó ńkọ́
6,ẹ má bínú ọ̀gá j́alẹ̀jálẹ̀ ni mo ti sọ fún yín.,áàá èmi tí mo ti jẹ́ oníbàárà yín tipẹ́. ó yẹ ...
7,ẹ má bínú ọ̀gá ṣùgbọ́n n kò lè fún yíin ní ẹ̀d...,ó dáa. màá fún ẹ ní ẹgbẹ̀rún méje náírà 7000 ...
8,ẹ fọkànbalẹ̀. ẹ̀yin náà sáà mọ̀ wá. adùn ti wa...,ó dáa. màá wá ní aago méjì. mo fẹ́ kí ṣọnà òdò...
9,ìyẹn kò ṣòro. ẹ máa san ìdajì owó ní àsansílẹ̀.,áàá ṣebí oníbàárà tipẹ́tipẹ́ ni mí. ẹ ò lè fọk...


In [11]:
#df['Answer'] = df['Answer'].astype(str).apply(preprocess)

In [12]:
#Tokenization using ByteLevelBPETokenizer

In [13]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, processors

In [14]:
all_texts = df['Question'].tolist() + df['Answer'].tolist()
#Initialize a byte pair encoding (BPE) Tokenizer
tokenizer =Tokenizer(models.BPE())
#Set the pre-tokenizer
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
#Initialize a trainer for the BPE model
trainer = trainers.BpeTrainer(special_tokens = ['<pad>','<s>','</s>','<unk>','<mask>'])
#Train the tokenizer on the dataset
tokenizer.train_from_iterator(all_texts,trainer)

#Tokenize the dataset
df['Question'] = df['Question'].apply(lambda x: tokenizer.encode(x).tokens)
df['Answer'] = df['Answer'].apply(lambda x: tokenizer.encode(x).tokens)

In [15]:
df

,Question,Answer
0,"[ẹ, káàbọ̀, sà, ., kí, ni, mo, lè, ṣe, fún, yín]","[mo, fẹ́ẹ́, ra, àkàràòyìnbó, fún, ọjọ́ìbí, ni]"
1,"[ó, dáa, ., onírú, èròja, lẹ, fẹ́ẹ́, rà]","[onípele, méjì, ti, ó, ní, èròjà, ‘, strawberr..."
2,"[ṣé, àkàràòyìnbó, aláìlẹ́yin, lẹ, fẹ́]","[bẹ́ẹ̀ni, aláìlẹ́yin, ni, mo, fẹ́, .]"
3,"[ó, dáa, ., ṣùgbọn, ẹni, tó, bá, béèrè, ní, a,...","[kò, burú, ., màá, padà, wá, títí, aago, méjì,..."
4,"[báwo, ni, kí, àkàràòyìnbó, náà, tóbi, tó]","[oníkílò, mẹ́ta, .]"
...,...,...
204,"[ó, dáa, ẹ, pò, ó]","[ó, dáa, .]"
205,"[eélòó, ni, àpò, ilá]","[ọgọ́rùnún, mẹ́fà, náírà, n600, ni, à, ń, ta, ..."
206,"[ọgọ́rùnún, mẹ́fà, náírà, n600, ló, bá, dé, lá...","[ẹ, jẹ́, kí, a, san, ogọ́rùnún, márùnún, àti, ..."
207,"[àpò, mélòó, ni, ẹ, fẹ́]","[àpò, kan, péré, ni, .]"


In [16]:
#Encoding using Byte Pair Encoding

In [17]:
def encode_text(text):
    return tokenizer.encode(text).ids

df['Question'] = df['Question'].astype(str).apply(encode_text)
df['Answer'] = df['Answer'].astype(str).apply(encode_text)

In [18]:
df

,Question,Answer
0,"[54, 822, 247, 5, 99, 62, 125, 176, 128, 97, 208]","[125, 403, 117, 348, 97, 916, 62]"
1,"[46, 90, 5, 790, 863, 279, 403, 64]","[793, 102, 77, 46, 73, 862, 56, 932, 57, 62]"
2,"[110, 348, 433, 279, 116]","[173, 433, 62, 125, 116, 5]"
3,"[46, 90, 5, 808, 720, 170, 124, 877, 73, 17, 1...","[130, 876, 5, 162, 292, 126, 522, 302, 102, 21..."
4,"[772, 62, 99, 348, 179, 802, 170]","[792, 252, 5]"
...,...,...
204,"[46, 90, 54, 218, 46]","[46, 90, 5]"
205,"[91, 62, 234, 616]","[92, 411, 69, 342, 62, 39, 49, 88, 234, 5]"
206,"[92, 411, 69, 342, 182, 124, 351, 402, 656, 21...","[54, 134, 99, 17, 149, 654, 163, 107, 113, 69,..."
207,"[234, 313, 62, 54, 116]","[234, 105, 295, 62, 5]"


In [19]:
# Vectorization using pytorch embedding layer
#Find the size of the vocabulary
vocab_size = tokenizer.get_vocab_size()


#Set the embedding dimension
embedding_dim = 50

#create an embedding layer
embedding = nn.Embedding(vocab_size, embedding_dim)

#function to convert token IDS to vectors
def tokens_to_vectors(token_ids):
    token_tensor = torch.tensor(token_ids)
    return embedding(token_tensor).detach().numpy()

#Convert token IDS to vectors for the dataset, then we have question vectors and answers vectors
df['Question'] = df['Question'].apply(tokens_to_vectors)
df['Answer'] = df['Answer'].apply(tokens_to_vectors)
df

,Question,Answer
0,"[[0.4095842, -2.6470711, -0.23661102, -2.41063...","[[-1.0868267, -0.84100384, -0.51391405, 0.9527..."
1,"[[-0.5916155, -1.5135745, 1.4694152, -0.386092...","[[0.7044864, -0.14969997, 0.65647537, -0.36956..."
2,"[[0.046047244, 1.8074447, -0.83933645, 0.51329...","[[0.6109264, -2.20843, 0.1453653, -0.6884896, ..."
3,"[[-0.5916155, -1.5135745, 1.4694152, -0.386092...","[[1.3218514, 0.27715072, 0.15183333, -1.233852..."
4,"[[0.107252896, 0.90961677, -0.4794056, 0.21072...","[[0.8070534, 1.8897266, -1.6924546, -0.1444083..."
...,...,...
204,"[[-0.5916155, -1.5135745, 1.4694152, -0.386092...","[[-0.5916155, -1.5135745, 1.4694152, -0.386092..."
205,"[[0.034666322, -0.43855482, -0.7957363, 0.8311...","[[1.6350237, 0.17146409, 0.23065408, -0.677863..."
206,"[[1.6350237, 0.17146409, 0.23065408, -0.677863...","[[0.4095842, -2.6470711, -0.23661102, -2.41063..."
207,"[[0.12017957, 0.4839849, -2.0559475, -0.397717...","[[0.12017957, 0.4839849, -2.0559475, -0.397717..."


In [20]:
#Dataset Split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df['Question'], df['Answer'], test_size=0.2
                                                   )

In [22]:
print("Number of Training Sample: ",X_train.shape)
print("Number of Training label: ",y_train.shape)
print("Number of Test Sample: ",X_test.shape)
print("Number of Test label: ",y_test.shape)

Number of Training Sample:  (167,)
Number of Training label:  (167,)
Number of Test Sample:  (42,)
Number of Test label:  (42,)


In [23]:
#Model Training

In [ ]:
#Model Evaluation